<a href="https://colab.research.google.com/github/anac777/climate/blob/main/Climate_AT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Goal: Compare the oustanding amounts of sustainable debt securities issued by Portugal, as published by the ECB and Banco de Portugal.**

> ECB source: https://data.ecb.europa.eu/data/datasets/CSEC

> Banco de Portugal source: https://bpstat.bportugal.pt/dominios/194/

📌 Methodology:
- Data retrieved directly from ECB Data Portal and BPstat APIs
- Series matched using predefined correspondence table
- Monthly observations aligned on common periods only
- Differences computed as:
    * Absolute difference: BPstat − ECB
    * Percentage difference: relative to ECB (reference dataset)
- Outliers and scale effects handled at visualization level

📌 Metadata:
* Frequency: Monthly
* Reference area: Portugal
* Counterpart area: World
* Reference sector: Total economy
* Counterpart sector: Total economy
* Maturity: All original maturities
* Unit: Millions of Euro
* Valuation: Market value

In [4]:
# 1. Import the necessary packages
# _________________________________
import pandas as pd
import requests
from io import StringIO
from pyjstat import pyjstat
import plotly.graph_objects as go
import plotly.express as px

print("All packages installed correctly")

All packages installed correctly


In [5]:
# 2. Define the BPstat download function via API
# ______________________________________________

BPSTAT_API_URL = "https://bpstat.bportugal.pt/data/v1"

def download_bpstat_series(series_ids):
    all_df = []

    for series_id in series_ids:
        try:

            url = f"{BPSTAT_API_URL}/series/?lang=EN&series_ids={series_id}"
            series_info = requests.get(url).json()[0]

            domain_id = series_info["domain_ids"][0]
            dataset_id = series_info["dataset_id"]

            # Dataset URL
            dataset_url = f"{BPSTAT_API_URL}/domains/{domain_id}/datasets/{dataset_id}/?lang=EN&series_ids={series_id}"

            dataset = pyjstat.Dataset.read(dataset_url)
            df = dataset.write('dataframe')

            # Detect date column dynamically
            date_col = next((c for c in df.columns if c.lower() in ["period", "time_period", "date"]), None)
            if date_col is None:
                raise ValueError(f"No date column found for series {series_id}")

            df = df.rename(columns={date_col: "date", "value": "value"})
            df["date"] = pd.to_datetime(df["date"])
            df["series_id"] = str(series_id)

            all_df.append(df[["date", "value", "series_id"]])

        except Exception as e:
            print(f"Failed to download series {series_id}: {e}")

    return pd.concat(all_df, ignore_index=True) if all_df else pd.DataFrame(columns=["date", "value", "series_id"])


In [6]:
# 3. Define ECB download function via API
# _______________________________________

def download_ecb(series_code: str, start_period: str=None, end_period: str=None) -> pd.DataFrame:
    base = "https://data-api.ecb.europa.eu/service/data"
    db, series_key = series_code.split(".", 1)
    url = f"{base}/{db}/{series_key}"

    params = {"format": "csvdata"}
    if start_period:
        params["startPeriod"] = start_period
    if end_period:
        params["endPeriod"] = end_period

    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()

    df = pd.read_csv(StringIO(r.text))
    df = df.rename(columns={"TIME_PERIOD": "date", "OBS_VALUE": "value"})
    df["date"] = pd.to_datetime(df["date"])
    df["series_id"] = series_code
    return df[["date", "value", "series_id"]]

In [7]:
# 4. Series mapping (BPstat to ECB)
# _______________________________________

series_map = {
    "12834056": "CSEC.M.N.PT.W0.S1.S1.N.L.LE.F3.T._Z.EUR.EUR.M.V.N.G_XX",
    "12834141": "CSEC.M.N.PT.W0.S1.S1.N.L.LE.F3.T._Z.EUR.EUR.M.V.N.L_XX",
    "12834303": "CSEC.M.N.PT.W0.S1.S1.N.L.LE.F3.T._Z.EUR.EUR.M.V.N.S_XX",
    "12834314": "CSEC.M.N.PT.W0.S1.S1.N.L.LE.F3.T._Z.EUR.EUR.M.V.N.C_XX"
}

In [8]:
# 5. Define ECB download function via API
# _______________________________________

bpstat_series_ids = list(series_map.keys())
bpstat_df = download_bpstat_series(bpstat_series_ids)

# Download ECB series using the mapping
ecb_df_list = []
for bp_id, ecb_code in series_map.items():
    df = download_ecb(ecb_code, start_period="2019-01")
    # add a column to link with BPSTAT
    df["bpstat_series_id"] = bp_id
    ecb_df_list.append(df)
ecb_df = pd.concat(ecb_df_list, ignore_index=True)

# Rename ECB columns for clarity
ecb_df = ecb_df.rename(columns={"series_id": "ecb_series_id", "value": "value_ecb"})
bpstat_df = bpstat_df.rename(columns={"value": "value_bpstat"})

In [9]:
# 6.1. Consistency cheks: Missing values
# _______________________________________
print("BPstat missing values:")
print(bpstat_df.isna().sum())

print("\nECB missing values:")
print(ecb_df.isna().sum())


BPstat missing values:
date            0
value_bpstat    0
series_id       0
dtype: int64

ECB missing values:
date                0
value_ecb           0
ecb_series_id       0
bpstat_series_id    0
dtype: int64


In [10]:
# 6.2. Consistency cheks: Period coverage in both datasets
# _________________________________________________________
for df, name in [(bpstat_df, "BPSTAT"), (ecb_df, "ECB")]:
    print(f"{name} date ranges per series:")
    print(df.groupby("series_id" if name=="BPSTAT" else "bpstat_series_id")["date"].agg(["min", "max"]))


BPSTAT date ranges per series:
                 min        max
series_id                      
12834056  2019-01-31 2025-11-30
12834141  2019-01-31 2025-11-30
12834303  2019-01-31 2025-11-30
12834314  2019-01-31 2025-11-30
ECB date ranges per series:
                        min        max
bpstat_series_id                      
12834056         2020-12-01 2025-11-01
12834141         2020-12-01 2025-11-01
12834303         2020-12-01 2025-11-01
12834314         2020-12-01 2025-11-01


In [11]:
# 7.1. Time alignment: Convert dates to have the same format in both datasets
# ____________________________________________________________________________
bpstat_df["date"] = pd.to_datetime(bpstat_df["date"]).dt.strftime('%b%Y')
ecb_df["date"] = pd.to_datetime(ecb_df["date"]).dt.strftime('%b%Y')


In [12]:
# 7.2. Time alignment: Keep only common months
# _____________________________________________
common_dates = {}

for bp_id, ecb_code in series_map.items():
    bp_dates = set(bpstat_df[bpstat_df["series_id"]==bp_id]["date"])
    ecb_dates = set(ecb_df[ecb_df["bpstat_series_id"]==bp_id]["date"])
    common_dates[bp_id] = bp_dates & ecb_dates  # intersection

# Filter datasets to keep only common months
bpstat_df = pd.concat([
    df[df["date"].isin(common_dates[bp_id])]
    for bp_id, df in bpstat_df.groupby("series_id")
], ignore_index=True)

ecb_df = pd.concat([
    df[df["date"].isin(common_dates[bp_id])]
    for bp_id, df in ecb_df.groupby("bpstat_series_id")
], ignore_index=True)

In [13]:
# 8. Merge on date + series mapping
# __________________________________

merged_df = pd.merge(
    bpstat_df,
    ecb_df,
    left_on=["date", "series_id"],
    right_on=["date", "bpstat_series_id"],
    how="inner"
)

# Add series names to improve interpretability
names = {
    "12834056": "Green",
    "12834141": "Sustainability-linked",
    "12834303": "Sustainability",
    "12834314": "Social"
}

merged_df["series_name"] = merged_df["series_id"].map(names)

# Convert back to datetime for sorting, but keep mmmyyyy for display
merged_df["date_dt"] = pd.to_datetime(merged_df["date"], format='%b%Y')

In [14]:
# 9. Compute differences
# _______________________

merged_df["abs_diff"] = merged_df["value_bpstat"] - merged_df["value_ecb"]
merged_df["pct_diff"] = merged_df["abs_diff"] / merged_df["value_ecb"] * 100

# Reorder columns and sort data
merged_df = merged_df[[
    "date",
    "date_dt",
    "series_name",
    "value_bpstat",
    "value_ecb",
    "abs_diff",
    "pct_diff",
    "series_id",
    "ecb_series_id"
]]

merged_df = merged_df.sort_values(["series_name", "date_dt"]).reset_index(drop=True)

merged_df.groupby("series_name")[["abs_diff", "pct_diff"]].describe()

abs_diff                                              \
                         count       mean        std         min       25%   
series_name                                                                  
Green                     60.0  -0.231529  22.728317 -156.094764 -0.500034   
Social                    60.0   1.470965   2.638530   -3.642397  0.000000   
Sustainability            60.0   0.044699   0.556372   -1.819589 -0.001233   
Sustainability-linked     60.0  74.480938  75.424147   -2.618545 -0.033260   

                                                         pct_diff            \
                             50%         75%         max    count      mean   
series_name                                                                   
Green                   1.474964    6.364359   43.171802     60.0 -0.035869   
Social                  0.117911    2.948091    8.883355     50.0  0.157054   
Sustainability          0.005342    0.022312    1.753839     51.0  0.007280   
Sustainability-linked  52.685208  151.014765  154.907051     60.0  5.337060   

                                                                         \
                            std       min       25%       50%       75%   
series_name                                                               
Green                  0.610449 -4.593664 -0.006150  0.028752  0.079991   
Social                 0.248506 -0.364035  0.011499  0.021683  0.347684   
Sustainability         0.122397 -0.401622 -0.001795  0.002763  0.008788   
Sustainability-linked  7.500341 -0.209551 -0.016728  1.987992  7.459098   

                                  
                             max  
series_name                       
Green                   0.442580  
Social                  0.759854  
Sustainability          0.329393  
Sustainability-linked  28.207599

In [20]:
# 10. Interactive graph for the BPstat and ECB time series (choose category in dropdown menu)
# ___________________________________________________________________________________________

fig = go.Figure()

series_list = merged_df["series_name"].unique()
for series in series_list:
    df = merged_df[merged_df["series_name"] == series]
    fig.add_trace(go.Scatter(
        x=df["date_dt"],
        y=df["value_bpstat"],
        mode="lines+markers",
        name=f"{series} - BPSTAT",
        visible=True
    ))
    fig.add_trace(go.Scatter(
        x=df["date_dt"],
        y=df["value_ecb"],
        mode="lines+markers",
        name=f"{series} - ECB",
        visible=True
    ))

# Create dropdown buttons
n_series = len(series_list)
buttons = []

for i, series in enumerate(series_list):
    # Initialize all traces as invisible
    visibility = [False] * (n_series * 2)
    visibility[i*2] = True      # BPSTAT
    visibility[i*2+1] = True    # ECB
    buttons.append(dict(
        label=series,
        method="update",
        args=[{"visible": visibility},
              {"title": f"BPSTAT vs ECB - {series}"}]
    ))

# Add dropdown menu
fig.update_layout(
    updatemenus=[dict(active=0, buttons=buttons)],
    xaxis=dict(
        title="Month",
        tickvals=merged_df["date_dt"].unique(),
        ticktext=merged_df["date"].unique()
    ),
    yaxis_title="Million EUR",
    title="BPstat vs ECB: Series Comparison",
    legend_title="Series / Source",
    hovermode="x unified"
)

fig.show()

In [24]:
# 11. Interactive graph for the absolute differences between sources (select the series by clicking in the legend)
# ________________________________________________________________________________________________________________

df_melt = merged_df.melt(
    id_vars=["date_dt", "series_name", "date"],
    value_vars=["abs_diff"],
    var_name="Difference_Type",
    value_name="Value"
)

fig = px.line(
    df_melt,
    x="date_dt",
    y="Value",
    color="series_name",
    line_dash="Difference_Type",  # here only 'abs_diff', keeps structure consistent
    hover_data=["date", "series_name", "Value"],
    title="BPstat vs ECB: Absolute differences"
)

fig.update_xaxes(
    tickvals=merged_df["date_dt"].unique(),
    ticktext=merged_df["date"].unique(),
    title_text="Month"
)

fig.update_yaxes(title_text="Million EUR")

fig.update_layout(
    legend_title_text='Type of sustainable debt securities',
    hovermode="x unified"
)

fig.show()

In [31]:
# 12. Heatmap for the percentage differences between sources
# __________________________________________________________

# Ensure merged_df is sorted chronologically
merged_df = merged_df.sort_values("date_dt").reset_index(drop=True)

# Pivot: rows = months (datetime), columns = series, values = pct_diff
heatmap_df = merged_df.pivot(index="date_dt", columns="series_name", values="pct_diff")

# Create a display label column from datetime for mmmyyyy
heatmap_df["month_label"] = heatmap_df.index.strftime("%b%Y")

heatmap_melt = heatmap_df.reset_index().melt(
    id_vars=["date_dt", "month_label"],
    var_name="Series",
    value_name="Percentage Difference"
)

# Create heatmap
fig = px.density_heatmap(
    heatmap_melt,
    x="month_label",   # mmmyyyy for readability
    y="Series",
    z="Percentage Difference",
    color_continuous_scale="RdBu_r",
    text_auto=True,
    title="BPstat vs ECB: Percentage Differences"
)

fig.update_xaxes(
    title_text="Month",
    categoryorder='array',
    categoryarray=heatmap_melt.sort_values("date_dt")["month_label"].unique()
)
fig.update_yaxes(title_text="Type of sustainable debt securities")
fig.update_layout(coloraxis_colorbar=dict(title="Pct Diff (%)"))

fig.show()

## **Goal: Compare the oustanding amounts of sustainable debt securities issued by EA countries, as published by the ECB.**

> Source: https://data.ecb.europa.eu/data/datasets/CSEC

Values are for the latest period available (November 2025), so Bulgaria is not included.

In [25]:
# 1. Define the series and the ISO2 codes for EA countries
# __________________________________________________________

EA_COUNTRIES = [
    "AT","BE","CY","DE","EE","ES","FI","FR","GR","HR",
    "IE","IT","LT","LU","LV","MT","NL","PT","SI","SK"
]

ECB_SERIES_TEMPLATES = [
    "CSEC.M.N.{cc}.W0.S1.S1.N.L.LE.F3.T._Z.EUR.EUR.M.V.N.G_XX",
    "CSEC.M.N.{cc}.W0.S1.S1.N.L.LE.F3.T._Z.EUR.EUR.M.V.N.L_XX",
    "CSEC.M.N.{cc}.W0.S1.S1.N.L.LE.F3.T._Z.EUR.EUR.M.V.N.S_XX",
    "CSEC.M.N.{cc}.W0.S1.S1.N.L.LE.F3.T._Z.EUR.EUR.M.V.N.C_XX",
]

In [26]:
# 2. Get the series from the source
# __________________________________

all_data = []

for country in EA_COUNTRIES:
    for template in ECB_SERIES_TEMPLATES:
        series_code = template.format(cc=country)
        try:
            df = download_ecb(series_code)
            df["country"] = country
            df["series_code"] = series_code
            all_data.append(df)
        except Exception as e:
            print(f"Failed for {series_code}: {e}")

ecb_all_df = pd.concat(all_data, ignore_index=True)

In [27]:
# 3. Prepare dates and values, and keep only the last period availableet the series from the source
# __________________________________________________________________________________________________

# Detect date column
date_col = next(
    col for col in ecb_all_df.columns
    if col.lower() in ["time_period", "time", "date", "period"]
)

# Detect value column
value_col = next(
    col for col in ecb_all_df.columns
    if col.lower() in ["obs_value", "value"]
)

# Convert
ecb_all_df["date_dt"] = pd.to_datetime(ecb_all_df[date_col])
ecb_all_df["value"] = pd.to_numeric(ecb_all_df[value_col], errors="coerce")

latest_date = ecb_all_df["date_dt"].max()

latest_df = ecb_all_df[
    ecb_all_df["date_dt"] == latest_date
].copy()

In [28]:
# 4. Sum the 4 series per country, to obtain the total
# ______________________________________________________

country_totals = (
    latest_df
    .groupby("country", as_index=False)["value"]
    .sum()
)

country_totals.rename(columns={"value": "total_outstanding"}, inplace=True)

country_totals.style.format({"total_outstanding": "{:,.0f} M€".format})

,country,total_outstanding
0,AT,"37,588 M€"
1,BE,"58,390 M€"
2,CY,"1,552 M€"
3,DE,"313,499 M€"
4,EE,401 M€
5,ES,"114,195 M€"
6,FI,"31,426 M€"
7,FR,"441,776 M€"
8,GR,"7,921 M€"
9,HR,989 M€


In [29]:
# 5. From ISO2 to ISO3, needed to make the graph in the next steps
# ________________________________________________________________

iso2_to_iso3 = {
    "AT":"AUT","BE":"BEL","CY":"CYP","DE":"DEU","EE":"EST","ES":"ESP",
    "FI":"FIN","FR":"FRA","GR":"GRC","HR":"HRV","IE":"IRL","IT":"ITA",
    "LT":"LTU","LU":"LUX","LV":"LVA","MT":"MLT","NL":"NLD","PT":"PRT",
    "SI":"SVN","SK":"SVK"
}

country_totals["iso3"] = country_totals["country"].map(iso2_to_iso3)

In [30]:
# 6. Make the graph
# __________________

fig = px.choropleth(
    country_totals,
    locations="iso3",
    locationmode="ISO-3",
    color="total_outstanding",
    hover_name="country",
    color_continuous_scale="Blues",
    title=f"Outstanding amounts of sustainable debt securities issued by Euro Area countries ({latest_date.strftime('%b %Y')})"
)

fig.update_layout(
    geo=dict(
        scope="europe",
        projection_type="natural earth",
        showframe=False,
        showcoastlines=True
    ),
    coloraxis_colorbar=dict(title="Outstanding amount")
)

fig.show()